In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Drop_pre_1.csv to Drop_pre_1.csv


In [3]:
df = pd.read_csv('Drop_pre_1.csv')
df.head()

,total_logins,time_on_platform_hours,forum_posts,%_assignments_completed,avg_quiz_score,days_since_last_login,time_per_login,low_quiz_flag,inactive_flag,assignments_per_login,engagement_index,dropout
0,20,17.489207,4,49.614498,24.682498,38,0.509416,0,0,0.457127,41.718798,1
1,121,74.001119,6,91.093147,87.941225,2,1.741965,0,0,1.295710,107.813749,0
2,17,5.106964,0,28.907581,41.318311,34,0.866577,1,1,0.392016,31.490357,1
3,124,71.800019,11,99.753192,96.108220,4,0.687305,0,0,1.257312,125.144565,0
4,20,6.590066,2,17.720747,18.077526,17,0.429602,1,0,0.145705,22.319309,1


In [4]:
X = df.drop(['dropout'], axis=1)
y = df['dropout']

In [5]:
n_cols = X.select_dtypes(include='number').columns.tolist()
mean = X[n_cols].mean()
std = X[n_cols].std()
X[n_cols] = (X[n_cols] - mean) / std
X_s = X
X_s.head()

,total_logins,time_on_platform_hours,forum_posts,%_assignments_completed,avg_quiz_score,days_since_last_login,time_per_login,low_quiz_flag,inactive_flag,assignments_per_login,engagement_index
0,-0.792206,-0.548340,-0.602059,-0.441571,-1.107114,1.963824,-0.863865,-0.825678,-0.700048,-0.520457,-0.778685
1,1.453547,1.874938,-0.385977,0.958439,1.192123,-0.911409,1.734035,-0.825678,-0.700048,0.888552,0.663468
2,-0.858912,-1.079301,-1.034224,-1.140482,-0.502459,1.644354,-0.111060,1.210520,1.427758,-0.629858,-1.001864
3,1.520252,1.780553,0.154229,1.250737,1.488965,-0.751674,-0.488920,-0.825678,-0.700048,0.824036,1.041616
4,-0.792206,-1.015705,-0.818142,-1.518066,-1.347182,0.286605,-1.032092,1.210520,-0.700048,-1.043717,-1.201970


In [6]:
X_s_train = X_s.sample(frac=0.8, random_state=0)
X_s_test = X_s.drop(X_s_train.index)
y_train = y[X_s_train.index]
y_test = y.drop(X_s_train.index)


In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, w, y, b):
    m = X.shape[0]

    # Compute predictions
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)

    # Clip values to avoid log(0)
    y_hat = np.clip(y_hat, 1e-8, 1 - 1e-8)

    # Compute cost
    total_cost = - (1/m) * np.sum(
        y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)
    )

    return total_cost

In [8]:
def compute_gradients(X, w, y, b):
    m = X.shape[0]

    # Compute predictions
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)

    # Compute gradients
    dw = (1/m) * np.dot(X.T, (y_hat - y))
    db = (1/m) * np.sum(y_hat - y)

    return dw, db


In [9]:
def train_logistic_regression(X, y, lr, epochs):
    m, n = X.shape
    w = np.zeros(n)
    b = 0.0
    costs = []

    for epoch in range(epochs):
        # Compute gradients
        dw, db = compute_gradients(X, w, y, b)

        # Update parameters
        w -= lr * dw
        b -= lr * db

        # Compute cost every 100 epochs
        if epoch % 10000 == 0 or epoch == epochs - 1:
            cost = compute_cost(X, w, y, b)
            costs.append(cost)
            print(f"Epoch {epoch}: Cost = {cost:.6f}")

    return w, b, costs


In [10]:
learning_rate = 0.02
epochs = 100000

# Train on training data
w, b, costs = train_logistic_regression(X_s_train, y_train, lr=learning_rate, epochs=epochs)

Epoch 0: Cost = 0.656950
Epoch 10000: Cost = 0.001321
Epoch 20000: Cost = 0.000713
Epoch 30000: Cost = 0.000497
Epoch 40000: Cost = 0.000385
Epoch 50000: Cost = 0.000315
Epoch 60000: Cost = 0.000268
Epoch 70000: Cost = 0.000233
Epoch 80000: Cost = 0.000207
Epoch 90000: Cost = 0.000186
Epoch 99999: Cost = 0.000170


In [11]:
def predict_classes(X, w, b, threshold=0.4):
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)
    return (y_hat >= threshold).astype(int)


In [12]:
y_test_pred = predict_classes(X_s_test, w, b)


In [13]:
def compute_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)


In [14]:
accuracy_test = compute_accuracy(y_test, y_test_pred)
print("Test Accuracy:", accuracy_test)


Test Accuracy: 1.0


In [15]:
def confusion_matrix(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tp = np.sum((y_true == 1) & (y_pred == 1))
    return np.array([[tn, fp], [fn, tp]])

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", cm)


Confusion Matrix:
 [[196   0]
 [  0 204]]


In [16]:
def precision_recall_f1(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp = cm[0]
    fn, tp = cm[1]

    precision = tp / (tp + fp) if (tp + fp) else 0
    recall = tp / (tp + fn) if (tp + fn) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    return precision, recall, f1

prec, rec, f1 = precision_recall_f1(y_test, y_test_pred)
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")


Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [21]:
import pickle

# Collect all model parameters in a dictionary
model_params = {
    "w": w,
    "b": b,
    "mean": mean,
    "std": std,


}

# Save to pickle file
with open("logistic_model.pkl", "wb") as f:
    pickle.dump(model_params, f)

print("Model saved to logistic_model.pkl")


Model saved to logistic_model.pkl


In [22]:

# Load pickle file
with open("logistic_model.pkl", "rb") as f:
    model_params = pickle.load(f)

w = model_params["w"]
b = model_params["b"]
mean_ = model_params["mean"]
std_ = model_params["std"]


print("Model loaded successfully!")


Model loaded successfully!


In [27]:
def predict_prob(X, w, b):
    z = np.dot(X, w) + b
    return 1 / (1 + np.exp(-z))

# Example single sample
X_new = np.array([[60, 30, 5, 70, 65, 5, 1.2, 0, 0, 1.0, 150.0]])

# Standardize
X_std = (X_new - mean.values) / std.values

# Predict probability
prob = predict_prob(X_std, w, b).item()

# Classify safely
pred_class = int(bool(prob >= 0.4))

print("Probability:", prob)
print("Prediction:", pred_class)


Probability: 0.0020997888482290382
Prediction: 0


In [29]:
files.download('logistic_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>